<a href="https://colab.research.google.com/github/UMassIonTrappers/quantum-computing-labs/blob/main/Lab_04_RWA_Rabi_oscillations_and_detuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 550/650

## Introduction to Quantum Computing

Robert Niffenegger (rniffenegger@umass.edu )


In [ ]:
'''
If plots of the Bloch sphere are not working you may have to revert the matplotlib back to 3.4.0

Qutip doesn't work with matplotlib 3.5
Qiskit doesn't work with matplotlib 3.4.3

Requires uninstall and then install of matplotlib.__version__ == '3.4.0'

'''

try:
  import matplotlib
  print(matplotlib.__version__)
except ImportError:
  print('Install Matplotlib')

if matplotlib.__version__ > '3.4.0':
  print ("Need to downgrade matplotlib to work with Qutip and Qisket")
  !pip uninstall matplotlib -y
  !pip install matplotlib==3.4.0
  import matplotlib ;  print(matplotlib.__version__)

#QuTiP

All labs will be run in the colaboratory Jupyter notebook like this one.

To get started we first need to install QuTiP using 'pip' within Colab.

(Any library not native to Colab can be installed this way.)

In [ ]:
try:
  import qutip
except ImportError:
  print('Install Qutip')
  !pip install qutip


#Qiskit

IBM's Quantum Information Software Kit

In [ ]:
try:
  import qiskit
except ImportError:
  print('Install Qiskit')
  !pip install -q qiskit
  !pip install -q qiskit[visualization]
  !pip install -q git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

In [ ]:
'''
Kaleidoscope plots Bloch spheres for Qisket
'''
try:
  import kaleidoscope
except ImportError:
  print('Install kaleidoscope')
  !pip install -q kaleidoscope
import kaleidoscope; print(kaleidoscope.__version__)

### Initialize

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from numpy import pi, sqrt, sin, cos


In [ ]:

import qutip as qt
from qutip import *
from qutip.qip import *
from qutip import qeye, tensor, destroy, fock_dm


#Lab 4

# Physical parameters to create qubit gates

Obviously Qubits don't flip in time "$\pi$". We need to start adding physical parameters to our qubits to understand them better.

For example:
Naively if we want the qubit state to flip in a period of 100 us then we'll need coupling that is at a rate of 1/T. So the coupling rate will need to be 1/100us = 10kHz.


>Still, the units of the coupling will need to be in units of angular frequency (radians/second, not Hz) to create rotation so we'll need to include 2$\pi$ as well. Radians/second vs. periods/second can be  confusing. Try to keep track of 2π's carefully. Remember qubits evolve in real time [seconds] but must go in circles of 2π.

Remember, rotation of the qubit state is due to the time evolution of the wavefunction set by the time dependent Schrödinger equation:

### $ \mathrm{i} \hbar \frac{\partial}{\partial t} | \,\psi (t) \rangle = \hat{H} | \,\psi (t) \rangle $

The Hamiltonian ${\hat {H}}$ will  describes the energy of the system and the Time dependent Schrödinger equation sets how the states evolve due to the Hamiltonian.

If the Hamiltonian ${\hat {H}}$ is constant in time, then the Schrödinger equation has the solution.

## $ |\Psi (t)\rangle =e^{-i{\hat {H}}t/\hbar }|\Psi (0)\rangle$



## Spin in an applied magnetic field

Imagine we have an electron with a spin dipole moment in an applied external magnetic field $\vec B$. The direction of the field is arbitrary but will immediately become the 'Z' basis for electron. So we will convienently say that is applied along the 'Z' direction in realspace $\hat e_z$.

$\vec B = B_z \hat e_z $

Now the electron spin has an energy within this applied magnetic field that is is trying to minimize to reach it's ground state.

The Hamiltonian describing it's energy is:

$\hat H = -  \gamma~ \vec S ⋅ \vec B =  -  \gamma \hat S_z B_0 $

Where $\hat S_z$ is the spin Z projection operator defined as $ \hat S_z = \frac{ħ}{2}\hat σ_z$

The energies of the eigenvectors (up,down states) are:

$ \hat H|±⟩ = E_±|±⟩ =  ∓ \frac{γ B ħ}{2}|±⟩$

>$γ = -\frac{\mu_B g_S} {\hbar} ≡$ gyromagnetic ratio

>$\mu_B$ is the Bohr magneton and $g_S$ is the Land´e g-factor of the spin ( ~2)

The spin up and down states are the eigenvectors of the spin and their associated eigenvalues are the energies:

$ E_± = ± ~\mu_B~ B_z $

$ \Delta E  = (E_+ - E_-)  = 2~\mu_B~ B_z$

$ \Delta E  = \hbar \omega_z = h f_z$

$ f_z = 2 ~\mu_B~ B_z / h = 2.8\text{ MHz/Gauss} $

$\omega_z = (2 \pi) f_z = 2 ~\mu_B~ B_z / \hbar = (2\pi)\cdot 2.8\text{ MHz/Gauss}$

---

For a 1 Guass magnetic field:

$\omega_z = (2\pi)\cdot 2.8 \times 10^6\text{ [radians/second]}$

$f_z = 2.8 \times 10^6\text{ [Hz]}$


> Note: This is just the *linear* Zeeman splitting. At stronger fields quadratic effects must be taken into account.



In [ ]:
#Plot
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1,1)
plt.style.use('default')
fs=15

bfield =np.linspace(0,2.5 ,20) # in Gauss

omegaz = - 1.4 * bfield #MHz/Gauss

axes.plot(bfield, omegaz,'-b')
axes.plot(bfield, -omegaz,'-r')

axes.set_xlabel('B [Gauss]', fontsize=fs)
axes.set_ylabel('Zeeman Splitting [MHz]', fontsize=fs);
plt.title('Zeeman splitting vs. Magnetic Field', fontsize=fs)
axes.legend( ("Down -1/2", "Up +1/2"));
plt.grid()

In [ ]:
from IPython.display import display, Math
import numpy as np
from numpy import pi, sqrt, sin, cos

'''
Setup input parameters
'''

omega_z = 2*pi* 2.8*10**6 # define rotation/coupling rate in rad/s

display(Math(r'\omega_z=2π*2.8*MHz/Gauss*1Gauss= {} [radian/sec] '.format(round(omega_z,3))))

# period = 2*pi/omega_z # period of rotating in seconds (not seconds/radian)
# print('Period=',round(period*1e6,3),'microseconds')



The direction of the applied external field defines the axis of the spin states, which we have referred to as the 'Z' direction on our Bloch sphere, so we can set the external field along 'z' in real-space as well.

$\hat H_0 = \frac{\mu_B g_S B_z}{2} \hat \sigma_z$

So the Hamiltonian creates rotations about the Z axis.

But how fast? We can check by solving for the rate of the rotation given by the solution to the time dependent Schrodinger Equation (TDSE), using $\theta = \omega_z t $.  

> $ |\Psi (t)\rangle =e^{-i{\hat {H}}t/\hbar }|\Psi (0)\rangle$

> $R_{z}(\theta )=e^{(-i\theta Z/2)}=\cos(\theta /2)I-i\sin(\theta /2)Z={\begin{bmatrix}e^{-i\theta /2}&0\\0&e^{i\theta /2}\end{bmatrix}}$

###$e^{-i\frac{{\hat H}}{ħ} t } = e^{-i \frac{\omega_z~\hat\sigma_z }{2}~t} = R_z (\omega_z t)$

Therefore
$\hat H = −\frac{\mu_B  g_S B_z}{\hbar}\hat S_z= −\frac{\mu_B  g_S B_z}{2}\hat \sigma_z =- \frac{\hbar  \omega_\pm }{ 2} \hat \sigma_z $


So each state of the qubit is rotating about the Z axis at a rate of $\omega_\pm$, which is dependent on the applied magnetic field.

$\omega_\pm = \frac{\mu_B  g_S B_z}{ħ} $


In [ ]:
# Initialize a superposition state so we can see the rotation
psi0 = spin_state( 1/2, +1/2) + spin_state( 1/2, -1/2)
psi0=psi0.unit()

H = 1/2*omega_z*sigmaz()  # setup the Hamiltonian

# List of times for which the solver should store the state vector
times = np.linspace(0, 2 * period, 50) # 2 periods of rotation at omega_z

'''
Run the 'simulation'
'''
result = sesolve(H, psi0, times, [])
# result = mesolve(H, psi0, times, [], [sigmaz(), sigmay(), sigmax()])

'''
Store the projection measurements for each basis
'''
#Project the state onto the Z axis (find the expectation values)
sz =  expect(sigmaz(), result.states)
sy =  expect(sigmay(), result.states)
sx =  expect(sigmax(), result.states)


In [ ]:
'''
Animate on the Bloch sphere
'''

#Plot setup
%matplotlib inline
from pylab import *
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

fig = plt.figure(figsize=(7, 7))
ax = Axes3D(fig,azim=-40,elev=30)
sphere = Bloch(axes=ax)
sphere.font_color = 'k'
sphere.vector_color = ['r','y','y','y']
sphere.point_color = ['r']

#Animation function
def animate(i):
    sphere.clear()
    sphere.vector_color = ['r','y','y','y']

    sphere.add_points([sx[:i+1],sy[:i+1],sz[:i+1]], meth='l') #Point for each step of the rotation
    sphere.add_vectors([sx[i],sy[i],sz[i]]) #Full vector from the simulation
    sphere.add_vectors([sx[i],0,0]) #Just the X basis component
    sphere.add_vectors([0,sy[i],0]) #Just the Y basis component
    sphere.add_vectors([0,0,sz[i]]) #Just the Z basis component
    sphere.make_sphere()
    return ax

def init():
    sphere.vector_color = ['r']
    return ax

ani = animation.FuncAnimation(fig, animate, np.arange(len(sx)),init_func=init, blit=False, repeat=True)
###ani.save('bloch_sphere.mp4', fps=20)

### Uncomment to create animation!
HTML(ani.to_jshtml())


In [ ]:
from pyparsing import hexnums
'''
Setup input parameters
'''
psi0 = spin_state( 1/2, 1/2) # Initialize the 'down' state

#Arbitrarily use 'unitless' rotation for coupling
Omega_Rabi = 1 # define rotation/coupling rate in rads/s (1 rad/s)

#Arbitrarily use 'unitless' rotation for qubit rotation
omega_z = 1 # define rotation/coupling rate in rads/s (1 rad/s)

Hx = 1/2*Omega_Rabi*sigmax()  # Hamiltonian for x rotation

Hz = 1*omega_z*sigmaz() # Hamiltonian for z rotation

# List of times for which the solver should store the state vector
times = np.linspace(0, pi/2, 25)

'''
Run the 'simulation'
'''
# Beam splitter #1
result = sesolve(Hx, psi0, times, [])
sz =  expect(sigmaz(), result.states)
sy =  expect(sigmay(), result.states)
sx =  expect(sigmax(), result.states)

# Pi rotation about z
result = sesolve(Hz, result.states[-1], times, [])
sz = np.append(sz, expect(sigmaz(), result.states))
sy = np.append(sy, expect(sigmay(), result.states))
sx = np.append(sx, expect(sigmax(), result.states))

# Beam Splitter #2
result = sesolve(Hx, result.states[-1], times, [])
sz = np.append(sz, expect(sigmaz(), result.states))
sy = np.append(sy, expect(sigmay(), result.states))
sx = np.append(sx, expect(sigmax(), result.states))

'''
Animate on the Bloch sphere
'''
#Plot setup
%matplotlib inline
from pylab import *
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

fig = plt.figure(figsize=(7, 7))
ax = Axes3D(fig,azim=-40,elev=30)
sphere = Bloch(axes=ax)
sphere.font_color = 'white'
sphere.vector_color = ['b','y','w','y']
sphere.point_color = ['y']

#Animation function
def animate(i):
    sphere.clear()
    sphere.vector_color = ['b','y','w','w']
    sphere.add_vectors([1,0,0]) #Add vector on X axis representing rotation axis

    sphere.add_points([sx[:i+1],sy[:i+1],sz[:i+1]], meth='l') #Point for each step of the rotation
    sphere.add_vectors([sx[i],sy[i],sz[i]]) #Full vector from the simulation
    sphere.add_vectors([0,sy[i],0]) #Just the Y basis component
    sphere.add_vectors([0,0,sz[i]]) #Just the Z basis component
    sphere.make_sphere()
    return ax

def init():
    sphere.vector_color = ['r']
    return ax

# Create the animation!
ani1 = animation.FuncAnimation(fig, animate, np.arange(len(sx)),init_func=init, blit=False, repeat=True)
HTML(ani1.to_jshtml())

# Rabi Oscillations

To flip the spin we need to apply an orthogonal magnetic field.
This will be a perturbation.

The large static magnetic field B can be applied by a pair of Helmholtz coils along $e_z$ and the oscillating magnetic field along $\hat e_x$ ($B_{RF}$) which can be applied by a small loop antenna a few centimeters above the electron/atom.

The additional small magnetic field from the radiation is time dependent.

$\vec B(t) = B_z \hat e_z +  B_{RF} ~cos ( \omega t ) \hat e_x $

Where the applied magnetic field amplitude can be determined by the power
output of the coil and the distance to the atoms (assuming an electrically small loop,
i.e. coil size much smaller than the wavelength).
$B_{RF} = 2 \sqrt{ \frac { \mu_0 }{c}\frac{P}{4\pi r^2}}$

> $\frac{P}{4\pi r^2}$ is defined as the 'irradiance' of the radiation as it spreads out in a sphere from the small antenna. It is the RF power spread over this areas so it is in units of Watts / meter.

The Hamiltonian now has two terms:

$\hat H = \hat H_0 +  \hat H_{int}$

$H_{int} =  −\frac{\mu_B  ⋅  g_S ⋅ B_{RF}}{2} ⋅  cos ( \omega t) \hat \sigma_x $

Because the magnetic field is applied perpendicular to the direction of the larger (polarizing and eigenstate creating field) it can create transitions between spin up and spin down.

Given that we suspect this Hamiltonian will also create rotations (albeit about a different axis) it is convenient to also give it a rate of rotation, now capital Omega, $Ω_{Rabi}$ which is a rate in angular frequency of rotation.

So before, the spin precession frequency was:  $\omega_z = \frac{\mu_B  ⋅  g_S ⋅  B_z}{\hbar} $ ,

now the Rabi Coupling is : $\Omega_{Rabi} = - \frac{\mu_B  ⋅  g_S ⋅  B_{RF}}{\hbar}  $

#$\hat H_{int} =  \frac{\Omega_{Rabi}}{2} cos ( \omega \cdot t) \hat \sigma_x $



In [ ]:
'''
Calculate the Rabi coupling rate from first principles
'''
#Import physics constants
from scipy import constants as const
g_s = 2 # Land´e g-factor of the spin ( ~2)

power = 0.05 # applied RF power in Watts ( 1 W )
r = 0.02 # Distance from antenna in meters ( 10 cm )

#Irradiance from the antenna
irradiance = power/(4*pi*r**2)
print('Irradiance = ', round(irradiance,3) , 'W/m^2')

# Magnetic field from the antenna which drives the spin transitions [in units of Teslas]
B_RF = 2* sqrt( const.mu_0 / const.c * irradiance)
print('B_RF drive =', round(B_RF,10), '[Tesla]')

#Omega from the magnetic field
Omega_Rabi = const.value('Bohr magneton')*g_s*B_RF / const.hbar

Omega_Rabi_f = Omega_Rabi / ( 2 * pi ) # Converting from rad/s to 1/s (Hz)
period_Omega_Rabi_f = 1 / Omega_Rabi_f # Calculating the period of a full oscillation in seconds

print('Rabi coupling = ', round(Omega_Rabi_f/1e3,2),'[kHz]')

print('Period of Rabi flopping = ', round(period_Omega_Rabi_f*1e6,2) ,'[us]')

In [ ]:
period_Omega_Rabi_f/2*sqrt(2)

## Rotating Wave Approximation (RWA)

Now that we have constructed the full Hamiltonian for the system we will perform a common trick to help describe its dynamics, a transformation to a rotating frame of reference. In this particular case the Hamiltonian has a time dependent component along the y-direction and a time-independent component along the z-direction. We would like to simplify the time-dependent component, so we will transform to a frame of reference rotating about z at frequency ω which is similar to the Zeeman splitting. We will also require that the time dependence of the RF field ($\omega_{RF}$ is similar to the Zeeman splitting (and rotation). So that $\omega  ∼ \omega_{RF}$ (but not equal).

$R_{z}(\omega t )=e^{-i\omega t Z/2}$

Appling this to the Hamiltonian

$R \hat H R^†$

>Excellent animations here: https://en.wikipedia.org/wiki/Unitary_transformation_(quantum_mechanics)#Rotating_frame

Resonant drive in the lab frame:

<!-- ![https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Resonant_lab.gif/330px-Resonant_lab.gif](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Resonant_lab.gif/330px-Resonant_lab.gif)
 -->


Following the derivation from lecture we get that:

$H_{rotating} = \frac{ħ δ}{2} \hat \sigma_z +  \frac{ħ Ω_{Rabi}}{2} \hat \sigma_x =\frac{ħ}{2} {\begin{pmatrix} δ & Ω_{Rabi} \\ Ω_{Rabi} & -δ \end{pmatrix}},$

In which we have transformed the Hamiltonian to a frame rotating at the same rate as the time dependence of the applied small B field we want to use to flip the spin.

In the rotating frame the two states now have a phase accumulation given by the detuning δ between their energy and the rotating frame (freq of applied RF field).

$ω_z≡$ Zeeman splitting from $B_z$

$ω_{rf}≡$ Frequency of applied drive radiation (smaller $B_{RF}$ from RF radiation)

$δ = ω_z - ω_{RF}$

If the freq $ω_{RF}$ of the applied drive field (and the rotating frame) is the same as the rate of rotation $ω_z$ from the $B_z$ then the detuning is zero $(δ = 0)$ and the frame is rotating at the same rate as the qubits.

# $Ω_{Rabi}$ is NOT $ω_z$ or $ω_{rf}$

$Ω_{Rabi}$ is the frequency of the coupling between the two states.

$Ω_{Rabi}$ sets the Rabi period of the Rabi oscillations between the states when an applied RF field drives the transitions and the qubit rotates about X or Y from |0⟩ to |1⟩ and back.

In [ ]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KPrSPqfVJhA?si=uVBdC2PTE2oubW8i" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

https://www.youtube.com/embed/KPrSPqfVJhA?si=uVBdC2PTE2oubW8i


A stroboscope helps us percieve the 'rotating frame' that the qubit and applied radition are in. Any offsets in the 'detuning' result in a residual phase accumulation...

Analogies:

Qubit = spinning wheel

Applied radiation = strobe light

Detuning = Residual apparent motion of line

# Rabi oscillation on resonance

In [ ]:
from IPython.display import display, Math

omega_z = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
print('0.7 MHz/Gauss')
display(Math(r'\omega_z=2π*0.7*MHz/Gauss*1Gauss= {} [MHz] '.format(round(omega_z/1e6/2/pi,3))))

#Match your applied rotation frequency to the qubit
omega_rf = omega_z

#Residual is the detuning
detuning = omega_z - omega_rf

display(Math(r'\delta= {}  [Hz]'.format(round(detuning/2/pi,3))))

display(Math(r'\omega_rf= {} [kHz]'.format(round(omega_rf/2/pi/1e3,2))))



In [ ]:
#Define the Rabi coupling

#set the desired period of Rabi Flops
period_Rabi = 100e-6 #100 microseconds
Omega_Rabi = 2*pi/period_Rabi # period of rotating in seconds (not seconds/radian)

print('Rabi Period=',round(period_Rabi*1e6,3),'[us]')
# display(Math(r'Period= {} ~milliseconds'.format(round(period*1e3,3))))

print('Rabi Frequency')
display(Math(r'\Omega_R= {} [kHz]'.format(round(Omega_Rabi/2/pi/1e3,2))))


In [ ]:
# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

H = 1/2*detuning*sigmaz()  + 1/2 * Omega_Rabi * sigmax()# setup the Hamiltonian

# List of times for which the solver should store the state vector
times = np.linspace(0, 2*period_Rabi, 50) # 2 periods of rotation at omega

'''
Run the 'simulation'
'''
result = sesolve(H, psi0, times, [])
# result = mesolve(H, psi0, times, [], [sigmaz(), sigmay(), sigmax()])

'''
Store the projection measurements for each basis
'''
#Project the state onto the Z axis (find the expectation values)
sz =  expect(sigmaz(), result.states)
sy =  expect(sigmay(), result.states)
sx =  expect(sigmax(), result.states)


In [ ]:
'''
Animate on the Bloch sphere
'''

#Plot setup

fig = plt.figure(figsize=(7, 7))
ax = Axes3D(fig,azim=-40,elev=30)
sphere = Bloch(axes=ax)
sphere.font_color = 'k'
sphere.vector_color = ['r','y','y','y']
sphere.point_color = ['r']

#Animation function
def animate(i):
    sphere.clear()
    sphere.vector_color = ['r','y','y','y']

    sphere.add_points([sx[:i+1],sy[:i+1],sz[:i+1]], meth='l') #Point for each step of the rotation
    sphere.add_vectors([sx[i],sy[i],sz[i]]) #Full vector from the simulation
    sphere.add_vectors([sx[i],0,0]) #Just the X basis component
    sphere.add_vectors([0,sy[i],0]) #Just the Y basis component
    sphere.add_vectors([0,0,sz[i]]) #Just the Z basis component
    sphere.make_sphere()
    return ax

def init():
    sphere.vector_color = ['r']
    return ax

ani = animation.FuncAnimation(fig, animate, np.arange(len(sx)),init_func=init, blit=False, repeat=True)
###ani.save('bloch_sphere.mp4', fps=20)

### Uncomment to create animation!
HTML(ani.to_jshtml())


In [ ]:
psi_down = spin_state( 1/2, -1/2)
psi_up = spin_state( 1/2, +1/2)

#Extract the expectation values of being in each state
prob_up =  expect(psi_up*psi_up.dag(), result.states)
prob_down =  expect(psi_down*psi_down.dag(), result.states)

#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')
plt.grid()

ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations on resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);

What if the detuning is off?



# Rabi oscillation OFF resonance (detuned)

In [ ]:
from IPython.display import display, Math

omega_z = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
print('0.7 MHz/Gauss')
display(Math(r'\omega_z=2π*0.7*MHz/Gauss*1Gauss= {} [kHz] '.format(round(omega_z/1e3/2/pi,3))))


#10kHz detuning
omega_rf = omega_z - 2*pi*10e3
display(Math(r'\omega_rf= {} [kHz]'.format(round(omega_rf/2/pi/1e3,2))))

#Residual is the detuning
detuning = omega_z - omega_rf
display(Math(r'\delta= {}  [kHz]'.format(round(detuning/2/pi/1e3,3))))



In [ ]:
# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

H = 1/2*detuning*sigmaz()  + 1/2*Omega_Rabi * sigmax()# setup the Hamiltonian

# List of times for which the solver should store the state vector
times = np.linspace(0, 1 * period_Rabi, 50) # 2 periods of rotation at omega

'''
Run the 'simulation'
'''
result = sesolve(H, psi0, times, [])
# result = mesolve(H, psi0, times, [], [sigmaz(), sigmay(), sigmax()])

'''
Store the projection measurements for each basis
'''
#Project the state onto the Z axis (find the expectation values)
sz =  expect(sigmaz(), result.states)
sy =  expect(sigmay(), result.states)
sx =  expect(sigmax(), result.states)



In [ ]:
'''
Animate on the Bloch sphere
'''

#Plot setup
%matplotlib inline
from pylab import *
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

fig = plt.figure(figsize=(7, 7))
ax = Axes3D(fig,azim=-40,elev=30)
sphere = Bloch(axes=ax)
# sphere.font_color = 'k'
sphere.vector_color = ['r','y','y','y']
sphere.point_color = ['r']

#Animation function
def animate(i):
    sphere.clear()
    sphere.vector_color = ['r','y','y','y']

    sphere.add_points([sx[:i+1],sy[:i+1],sz[:i+1]], meth='l') #Point for each step of the rotation
    sphere.add_vectors([sx[i],sy[i],sz[i]]) #Full vector from the simulation
    sphere.add_vectors([sx[i],0,0]) #Just the X basis component
    sphere.add_vectors([0,sy[i],0]) #Just the Y basis component
    sphere.add_vectors([0,0,sz[i]]) #Just the Z basis component
    sphere.make_sphere()
    return ax

def init():
    sphere.vector_color = ['r']
    return ax

ani = animation.FuncAnimation(fig, animate, np.arange(len(sx)),init_func=init, blit=False, repeat=True)
###ani.save('bloch_sphere.mp4', fps=20)

### Uncomment to create animation!
HTML(ani.to_jshtml())


In [ ]:
#Extract the expectation values of being in each state
prob_up =  expect(psi_up*psi_up.dag(), result.states)
prob_down =  expect(psi_down*psi_down.dag(), result.states)

#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')
plt.grid()

ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);

#Hadamard gate

https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_gate

https://qiskit.org/textbook/ch-states/single-qubit-gates.html#3.-The-Hadamard-Gate--


##$H={\frac {1}{\sqrt {2}}}{\begin{bmatrix}1&1\\1&-1\end{bmatrix}}=\frac{1}{\sqrt {2}}[\hat {\sigma_x}+\hat {\sigma_z}]$

It is a rotation about both the X and Z axes simultaneously

Just like our detuned pulse if the detuning is equal to the Rabi coupling


$H = \frac{ħ δ}{2}\sigma_z + \frac{\hbar \Omega}{2} \sigma_x =  \frac{ħ}{2} {\begin{pmatrix} δ & Ω  \\\ Ω & -δ\end{pmatrix}}$

>(e.g. if $δ = Ω = 1$)

However the pulse length is different.

The generalized Rabi oscillation rate must take into account the rotation about both axes to calculate a full period of rotation.

Therefore the Generalized Rabi Oscillation rate is:
$\Omega_G = \sqrt{δ^2 + Ω^2}$
To get a full period we must have $\Omega_G*t = 2 π$ and therefore,
$\Omega_G t = 2 π = \sqrt{(δ⋅t)^2 + (Ω⋅t)^2}$.

Therefore the period is shorter by 1/$\sqrt{2} ≈ 0.7$.

However, for a Hadamard gate we only want half a period (π rotation).
An equivalent rotation about Y or X to create a superposition would be only a π/2 rotation.

Therefore, the Hadamard creates a superposition state slightly slower than a direct X or Y rotation but with a better 'landing' in that it starts and ends 'smoothly' and allows for some errors in the pulse time and freq etc.

Next, we'll plot the Rabi oscillations for various detunings to see this explicitly.


# Rabi oscillations vs. detuning

In [ ]:

omega_z = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
display(Math(r'\omega_0=2π*0.7*MHz/Gauss*1Gauss= {} [MHz]'.format(round(omega_z/1e6/2/pi,0))))


#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')

# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

# Set the Rabi coupling rate
Omega_Rabi = 2*pi*10e3

#Loop - to plot points along rotation arc
deltas = np.linspace(0, 2*pi*10e3, 21)
for i in range(len(deltas)):

  # Set detuning
  omega_rf = omega_z - deltas[i]
  detuning = omega_z - omega_rf
  display(Math(r'\delta= {} '.format(round(detuning/2/pi,2))))

  # Setup Hamiltonian
  H = 1/2*detuning*sigmaz()  + 1/2*Omega_Rabi * sigmax()

  period_Rabi = 2*pi/Omega_Rabi # period of rotating in seconds (not seconds/radian)

  # List of times for which the solver should store the state vector
  times = np.linspace(0, 2 * period_Rabi, 100) # 2 periods of rotation at omega


  #Integrated Schrodinger Equation
  result = sesolve(H, psi0, times, [])

  #Extract the expectation values of being in each state
  prob_up =  expect(psi_up*psi_up.dag(), result.states)
  prob_down =  expect(psi_down*psi_down.dag(), result.states)

  ax.plot(times*1e6, prob_up,'-r',alpha=1/(i/10+2))
  ax.plot(times*1e6, prob_down,'-b',alpha=1/(i/10+2))



# fig, ax = plt.subplots(1,1)
# plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()


fig, ax = plt.subplots(1,1)
plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants
Omega_0 = 2 * np.pi  # Resonant Rabi frequency (rad/s)
time_steps = 400  # Number of time points
detuning_range = np.linspace(-3, 3, 400)  # Detuning values
time = np.linspace(0, 5, time_steps)  # Time array

# Compute Rabi oscillations as a function of time and detuning
def rabi_probability(t, Delta):
    Omega = np.sqrt((Delta * Omega_0)**2 + Omega_0**2)  # Total Rabi frequency
    return np.sin(Omega * t / 2)**2

# Create a figure for the animation
fig, ax = plt.subplots()
X, Y = np.meshgrid(time, detuning_range)  # Swap X and Y for rotation
Z = rabi_probability(X.ravel(), Y.ravel()).reshape(Y.shape)  # Adjusted for 2D inputs

contour = ax.contourf(time, detuning_range, Z, levels=100, cmap="coolwarm")
ax.set_xlabel("Time (t)")
ax.set_ylabel("Detuning (Δ)")
ax.set_title("Rabi Oscillations")

# Animation update function
def update(frame):
    ax.clear()
    shifted_Y = detuning_range + frame * 0.1
    Z = rabi_probability(np.repeat(time[np.newaxis, :], len(detuning_range), axis=0), shifted_Y[:, np.newaxis])
    contour = ax.contourf(time, detuning_range, Z, levels=100, cmap="viridis")
    ax.set_xlabel("Time (t)")
    ax.set_ylabel("Detuning (Δ)")
    ax.set_title(f"Rabi Oscillations (Frame {frame})")
    return contour

# # Create animation
# ani = FuncAnimation(fig, update, frames=100, interval=50)

# Save or display animation
plt.show()


#Quantum Circuits

Now we can fully see that quantum circuits are all set rotations about various axes or combinations of axes with set intervals of rotation ( π, π/2, π/4, etc.)

For instance in the detuned case of Rabi oscillations we have rotation not just about X but also Z. We can use the arbitraty rotation gate from Qisket to do the same thing.


Rotation about multiple axes
https://qiskit.org/textbook/ch-states/single-qubit-gates.html#7.-The-U-gate--
u( x, y, z, qubit)

For a Hadmard gate this would be:
u( x = pi/2 , y= 0 , z=pi, qubit=0)


While this is equivalent to a Hadamard gate in the end result. It is not the exact form we showed. It is the combination of sequential rotations.

https://qiskit.org/textbook/ch-states/single-qubit-gates.html#3.-The-Hadamard-Gate--



In [ ]:
from qiskit import QuantumCircuit, transpile, assemble, Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector
from kaleidoscope import bloch_sphere

sim = QasmSimulator() #Set simulator to evolve state

# Create a Quantum Circuit acting on the q register
qc = QuantumCircuit(1)


# Apply rotation gates to qubit '0' (in units of pi)

## Rotation about X by pi
# qc.rx( pi , 0)

## Rotation about Z by pi/2
# qc.rz( pi/2 , 0)

'''
# Rotation about multiple axes
# https://qiskit.org/textbook/ch-states/single-qubit-gates.html#7.-The-U-gate--
# u( x, y, z, qubit)
'''
# To rotate about x AND z (like a detuned pulse) we need Rx=pi/2 and Rz=pi simultaneously
qc.u(pi/2, 0, pi, 0)

# The Hadamard gate does this by design
#https://qiskit.org/textbook/ch-states/single-qubit-gates.html#3.-The-Hadamard-Gate--
# qc.h(0)


# Draw the circuit
qc.draw('mpl')

In [ ]:
compiled_circuit = transpile(qc, sim) #Compile quantum circuit
qc.save_statevector() #Save the state vector after simulation

'''
Run the simulation
'''
job = sim.run(qc, shots=100)

#Plot final states on Bloch spheres
bloch_sphere(job.result().get_statevector(qc))

In [ ]:
'''
Qisket demo
'''
# !pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

# # Run the code in this cell to see the widget
# from qiskit_textbook.widgets import gate_demo
# gate_demo(gates='pauli+h')


#Exercises:

1. Animate the evolution on the Bloch sphere of a single Hadamard gate starting with an up state.
  * Animate starting with the down state
  * What two states does the Hadamard transform the Z basis states into?
  * What basis are they?

2. Animate the evolution on the Bloch sphere of a second Hadamard gate starting with the state from the previous Hadamard gate.

3. What gate can we insert between the two Hadamard gates to end up in the down state? (Starting from the up state)
  * What is the gate sequence?
  * Verify the circuit works using Qisket
  * Trace the path of the state vector on the Bloch sphere for this sequence of gates.

4. Plot the expectation value of each spin state during the evolution of 4 Hadamard gates.

5. Given an RF power of 50mW, a loop antenna 2cm from the qubit and an external magnetic field of 2 Gauss. Find the pulse parameters to create the following gates:
  * X gate ('NOT gate)
  * Z gate ( pi phase gate)
  * Hadamard gate

  ( Parameters needed: Length of time of the pulse, frequency of the pulse )

6. Create a frequency sweep. With a fixed pulse power and time vary the detuning and plot the expectation value of each spin. Your Y axis will be the population of each spin after the pulse and X will be the detuning. Start with a pulse that on resonance is an X gate from above.
  * Replot with a stronger RF pulse but with a reduced time such that it still creates an X gate on resonance.

In [ ]:
'''
Exercise 1
'''



In [ ]:
'''
Exercise 2
'''



In [ ]:
'''
Exercise 3
'''



In [ ]:
'''
Exercise 4
'''




In [ ]:
'''
Exercise 5
'''




End of Lab 3

$\Delta R={\begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}}+\begin{bmatrix}0&z&-y\\-z&0&x\\y&-x&0\end{bmatrix} \Delta \theta =I +A \,\Delta \theta $


Therefore, if

 R=$\begin{bmatrix}a&b&c\\d&e&f\\g&h&i\\\end{bmatrix}$

then

u = $\begin{bmatrix}h-f\\c-g\\d-b\\\end{bmatrix}$

In [ ]:

omega0 = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
display(Math(r'\omega_0=2π*0.7*MHz/Gauss*1Gauss= {} [MHz]'.format(round(omega/1e6/2/pi,0))))


#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')

# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

# Set the Rabi coupling rate
Omega = 2*pi*20e3

#Loop - to plot points along rotation arc
steps = 41
max_delta = 2*pi*50e3
deltas = np.linspace(-max_delta, max_delta, steps)
final_prob_up = np.linspace(0, 0, steps)

for i in range(len(deltas)):

  # Set detuning
  omega = omega0 - deltas[i]
  detuning = omega0 - omega
  display(Math(r'\delta= {} '.format(round(detuning/2/pi,2))))

  # Setup Hamiltonian
  H = 1/2*detuning*sigmaz()  + 1/2*Omega * sigmax()

  period = 1/10e3 # period of rotating in seconds (not seconds/radian)

  # List of times for which the solver should store the state vector
  times = np.linspace(0, 1/2 * period, 100) # 2 periods of rotation at omega


  #Integrated Schrodinger Equation
  result = sesolve(H, psi0, times, [])

  #Extract the expectation values of being in each state
  prob_up =  expect(psi_up*psi_up.dag(), result.states)
  final_prob_up[i] = prob_up[-1]
  prob_down =  expect(psi_down*psi_down.dag(), result.states)

  ax.plot(times*1e6, prob_up,'-r',alpha=1/(i/10+2))
  ax.plot(times*1e6, prob_down,'-b',alpha=1/(i/10+2))



# fig, ax = plt.subplots(1,1)
# plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()



fig, ax = plt.subplots(1,1)
plt.style.use('default')
ax.plot(deltas, final_prob_up,'-r')
ax.plot(deltas, 1-final_prob_up,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r'delta []', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()



In [ ]:

omega0 = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
display(Math(r'\omega_0=2π*0.7*MHz/Gauss*1Gauss= {} [MHz]'.format(round(omega/1e6/2/pi,0))))


#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')

# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

# Set the Rabi coupling rate
Omega = 2*pi*10e3

#Loop - to plot points along rotation arc
steps = 101
delta_max = 2*pi*100e3
deltas = np.linspace(-delta_max,delta_max , steps)
final_prob_up = np.linspace(0, 0, steps)

for i in range(len(deltas)):

  # Set detuning
  omega = omega0 - deltas[i]
  detuning = omega0 - omega
  # display(Math(r'\delta= {} '.format(round(detuning/2/pi,2))))

  # Setup Hamiltonian
  H = 1/2*detuning*sigmaz()  + 1/2*Omega * sigmax()

  period = 2*pi/Omega # period of rotating in seconds (not seconds/radian)

  # List of times for which the solver should store the state vector
  times = np.linspace(0, 1/2 * period, 50) # 2 periods of rotation at omega


  #Integrated Schrodinger Equation
  result = sesolve(H, psi0, times, [])

  #Extract the expectation values of being in each state
  prob_up =  expect(psi_up*psi_up.dag(), result.states)
  final_prob_up[i] = prob_up[-1]
  prob_down =  expect(psi_down*psi_down.dag(), result.states)

  ax.plot(times*1e6, prob_up,'-r',alpha=1/(i/10+2))
  ax.plot(times*1e6, prob_down,'-b',alpha=1/(i/10+2))



# fig, ax = plt.subplots(1,1)
# plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()


fig, ax = plt.subplots(1,1)
plt.style.use('default')
ax.plot(deltas/1e3/2/pi, final_prob_up,'-r')
ax.plot(deltas/1e3/2/pi, 1-final_prob_up,'-b')
plt.title('Qubit Resonance vs detuning sweep')
ax.set_xlabel(r'detuning [kHz]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()



In [ ]:

omega0 = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
display(Math(r'\omega_0=2π*0.7*MHz/Gauss*1Gauss= {} [MHz]'.format(round(omega/1e6/2/pi,0))))


#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')

# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

# Set the Rabi coupling rate
Omega = 2*pi*20e3

#Loop - to plot points along rotation arc
steps = 101
delta_max = 2*pi*100e3
deltas = np.linspace(-delta_max,delta_max , steps)
final_prob_up = np.linspace(0, 0, steps)

for i in range(len(deltas)):

  # Set detuning
  omega = omega0 - deltas[i]
  detuning = omega0 - omega
  # display(Math(r'\delta= {} '.format(round(detuning/2/pi,2))))

  # Setup Hamiltonian
  H = 1/2*detuning*sigmaz()  + 1/2*Omega * sigmax()

  period = 2*pi/Omega # period of rotating in seconds (not seconds/radian)

  # List of times for which the solver should store the state vector
  times = np.linspace(0, 1/2 * period, 50) # 2 periods of rotation at omega


  #Integrated Schrodinger Equation
  result = sesolve(H, psi0, times, [])

  #Extract the expectation values of being in each state
  prob_up =  expect(psi_up*psi_up.dag(), result.states)
  final_prob_up[i] = prob_up[-1]
  prob_down =  expect(psi_down*psi_down.dag(), result.states)

  ax.plot(times*1e6, prob_up,'-r',alpha=1/(i/10+2))
  ax.plot(times*1e6, prob_down,'-b',alpha=1/(i/10+2))



# fig, ax = plt.subplots(1,1)
# plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
# ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()


fig, ax = plt.subplots(1,1)
plt.style.use('default')
ax.plot(deltas/1e3/2/pi, final_prob_up,'-r')
ax.plot(deltas/1e3/2/pi, 1-final_prob_up,'-b')
plt.title('Qubit Resonance vs detuning sweep')
ax.set_xlabel(r'detuning [kHz]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()



In [ ]:

omega0 = 2*pi* 0.7*10**6 # define rotation/coupling rate in rad/s
display(Math(r'\omega_0=2π*0.7*MHz/Gauss*1Gauss= {} [MHz]'.format(round(omega/1e6/2/pi,0))))


#Plot results
fig, ax = plt.subplots(1,1)
plt.style.use('default')

# Initialize the state
psi0 = spin_state( 1/2, +1/2)
psi0=psi0.unit()

# Set the Rabi coupling rate
Omega = 2*pi*30e3

#Loop - to plot points along rotation arc
steps = 101
delta_max = 2*pi*100e3
deltas = np.linspace(-delta_max,delta_max , steps)
final_prob_up = np.linspace(0, 0, steps)

for i in range(len(deltas)):

  # Set detuning
  omega = omega0 - deltas[i]
  detuning = omega0 - omega
  # display(Math(r'\delta= {} '.format(round(detuning/2/pi,2))))

  # Setup Hamiltonian
  H = 1/2*detuning*sigmaz()  + 1/2*Omega * sigmax()

  period = 2*pi/Omega # period of rotating in seconds (not seconds/radian)

  # List of times for which the solver should store the state vector
  times = np.linspace(0, 1/2 * period, 50) # 2 periods of rotation at omega


  #Integrated Schrodinger Equation
  result = sesolve(H, psi0, times, [])

  #Extract the expectation values of being in each state
  prob_up =  expect(psi_up*psi_up.dag(), result.states)
  final_prob_up[i] = prob_up[-1]
  prob_down =  expect(psi_down*psi_down.dag(), result.states)

  ax.plot(times*1e6, prob_up,'-r',alpha=1/(i/10+2))
  ax.plot(times*1e6, prob_down,'-b',alpha=1/(i/10+2))



# fig, ax = plt.subplots(1,1)
# plt.style.use('default')
ax.plot(times*1e6, prob_up,'-r')
ax.plot(times*1e6, prob_down,'-b')
plt.title('Rabi Oscillations OFF resonance')
ax.set_xlabel(r't [us]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()


fig, ax = plt.subplots(1,1)
plt.style.use('default')
ax.plot(deltas/1e3/2/pi, final_prob_up,'-r')
ax.plot(deltas/1e3/2/pi, 1-final_prob_up,'-b')
plt.title('Qubit Resonance vs detuning sweep')
ax.set_xlabel(r'detuning [kHz]', fontsize=20)
ax.set_ylabel(r'Probability', fontsize=20);
ax.legend(("Up +1/2", "Down -1/2"),loc=1);
plt.grid()

